In [330]:
import numpy as np

telementary = 1
runNum = 0      ## Increment to utilise caching
k=100       ## assumed size of dataset
learningRate = 0.001

Taking in input shape

# Layer Class

In [331]:
class nlayer:
    id = 0
    shape = 1               ## Defines self dimension (1D)
    input_layers = []        ## store layer pointer
    weights = None          ## assuming all input activations are concatenated (sorted on layer ID).
    bias = np.array([])      ## store self biases
    activationFn = "linear"         ## store self activation function


    ## Caching
    
    #### store last activation as cache to speed up when multiple layers use this layer as input. So this is evaluated only once.
    cachedRun = -2      # runNum when cache was calculated, can be old
    ## cachedRun = -1 & isAdaptive = 0 is for input layers
    cacheValue = None
    
    ## Flag indicating if it was being evaluated.
    #### This can help in case of self loops, when a layer was being evaluated was evaluated again
    #  meaning one of this layer's input_layer has this layer as one of the inputs (called self-loop is a graph).
    #  In this situation, the last cached value of this layer will be returned.
    # this may be used to simulate LSTM Network.
    beingEvaluated = 0  

    ## Error variance
    #### Store absolute sum of errors in terms of array of sum per node in 1D np array

    ## supports changing widths & depths. Not suitable for inputs and outputs
    isDynamic = 1
    


    ## Methods
    def __init__(self, shape=1, inputLayers=[], isInput=0, setInputValues=[], activationFn="linear", isDynamic=0) -> None:
        self.shape = shape
        self.activationFn = activationFn
        self.bias = np.zeros(shape)
        self.input_layers = []  ## Clearing on reinitializing

        if(isDynamic==0):
            self.isDynamic = 0
        if(isInput):
            self.cachedRun = -1
            self.isDynamic = 0
            if(len(setInputValues) != 0):
                self.cacheValue = np.array(setInputValues)
        else:
            # generating random weights if given
            if(type(inputLayers) == type([])):
                if(len(inputLayers) != 0):
                    for layer in inputLayers:
                        self.addInputLayer(layer)
            else:
                print("inputLayers should be a List.")
                if(type(inputLayers) == type(nlayer(1))):
                    self.addInputLayer(inputLayers)

    def addInputLayer(self, newInputLayer):
        # check if it doesn't already exists
        for layr in self.input_layers:
            if(newInputLayer == layr):
                print("Layer already exists.")
                return -1

        self.input_layers.append(newInputLayer)
        ## DONE: Generate random weights
        generatedColumn = np.random.rand(self.shape, newInputLayer.shape)
        if(type(self.weights) == type(None)):
            self.weights = generatedColumn
        else:
            self.weights = np.concatenate((self.weights, generatedColumn), axis=1)

    def addWidth_to_Layer(self, addWidth):
        if(addWidth > 0):
            self.shape += addWidth
            self.bias = np.concatenate((self.bias, np.random.rand(addWidth)))
            
            ## generating new row of random weights
            generatedRow = np.random.rand(addWidth, self.weights.shape[1])
            self.weights = np.concatenate((self.weights, generatedRow))
        else:
            print("error, doesn't support decrease.")


    def applyActivationFn(self,rawActivation):
        if(self.activationFn == "linear"):
            return rawActivation

        if(self.activationFn == "relu"):
            return np.maximum(rawActivation, 0)
        if(self.activationFn == "softmax"):
            A = np.exp(Z) / sum(np.exp(Z))
            return A
 
    def applyDerivActivationFn(self, input):
        if(self.activationFn == "linear"):
            return 1
        if(self.activationFn == "relu"):
            return (input > 0)
            
      

    def getActivation(self):    ## return np array of activation of current layer
        ## beingEvaluated == 1 means the node was triggered by a loop in the network. Returning last value cached prevents infinite loops.

        
        if(self.cachedRun == runNum or self.cachedRun == -1 or self.beingEvaluated == 1):   ## if activation was already calculated for this run OR is an input layer
            if(telementary): 
                if(self.cachedRun == -1):
                    print("Provided input from cache")
                else:
                    print("Re-used Cached Value")
            return(self.cacheValue)
        else:
            ## compiling a numpy array of all activation values listed in input layer. 
            inputArr = np.array([])

            self.beingEvaluated = 1

            for layrIndx in range(len(self.input_layers)):
                inputArr = np.concatenate((inputArr, self.input_layers[layrIndx].getActivation()))


            self.beingEvaluated = 0

            # Checking if shape matches
            if(inputArr.shape[0] > self.weights.shape[1]):
                if(telementary): print("!!!SHAPE MISMATCH!!!")  
                ## Adjust matrix dimension & adding new random weights to match size
                generatedColumn = np.random.rand(self.weights.shape[0], (inputArr.shape[0] - self.weights.shape[1]))
                self.weights = np.concatenate((self.weights, generatedColumn), axis=1)


            elif(inputArr.shape[0] < self.weights.shape[1]):       ## input layer may have been removed causing weight matrix to be larger than inputs
                print("!! Input Layer smaller than expected. !!")
                print("inputArr.shape[0] =", inputArr.shape[0], "self.weights.shape[1] =", self.weights.shape[1])
                return -1
            
            rawActivation = np.matmul(self.weights, inputArr) + self.bias
            activation = self.applyActivationFn(rawActivation=rawActivation)

            self.cachedRun = runNum
            # self.cacheValue = activation          ## storing a pointer to activation calculated
            self.cacheValue = np.copy(activation)   ## duplicating array

            if(telementary): print("activation =", activation, "& cached")  

            return activation


    def correct_error(self, activation_error):
        # if(type(self.cacheValue)==type(None)):
        #     self.
        if(self.cachedRun >= 0):    ## check if is run before
            ## compiling a numpy array of all activation values listed in input layer. 
            inputArr = np.array([])
            self.beingEvaluated = 1
            layerLengths = []   ## Store each layer's length to distribute corrections to them later

            for layrIndx in range(len(self.input_layers)):
                layerLengths.append(self.input_layers[layrIndx].shape)
                inputArr = np.concatenate((inputArr, self.input_layers[layrIndx].getActivation()))
            self.beingEvaluated = 0


            inputArr2 = inputArr[np.newaxis]

            # if(len(inputArr.shape) == 1):       ## if array is 1D, convert to 2D to support Transpose.
            #     inputArrT = inputArr[np.newaxis].T
            # else:
            #     inputArrT = inputArr.T


            dZ = self.cacheValue - activation_error

            dW = (1/k)*np.matmul(dZ, inputArr2)
            dB = (1/k)*np.sum(dZ)

            oldWeights = self.weights
            ## Updating self weights & biases
            self.weights = self.weights - learningRate*dW
            self.bias = self.bias - learningRate*dB

            ## Finding errors for input layers
            # dIZ = np.matmul(np.transpose(self.weights),dZ)
            dIZ = np.matmul((oldWeights.T), dZ) * self.applyDerivActivationFn(inputArr)

            ## Splitting input corrections to their corresponding layers
            splitPoints = [0]
            lengthTillNow = 0
            for layerIndx in range(len(layerLengths)):
                lengthTillNow += layerLengths[layerIndx]
                splitPoints.append(lengthTillNow)

                self.input_layers[layerIndx].correct_error(dIZ[splitPoints[-2]:splitPoints[-1]])

            return [self.cacheValue]

                



# Network Class

In [332]:
class network:
    input_shape=1  # Currently only 1D
    output_shape=1 # Currently only 1D
 
    input_layer = None      ## Pointer to input nlayer
    output_layer = None     ## Pointer to output nlayer

    layers = []
    numberOfLayers = 0      ## used to assign ID to new layer in matrix

    adaptive = 1

    def __init__(self, input_shape, output_shape, insertDefault=0) -> None:
        self.input_shape = input_shape
        self.output_shape = output_shape

        # Connect output with 1 adaptive neuron input
        self.input_layer = nlayer(input_shape, isInput=1)

        if(insertDefault==1):
            hiddenLayer = nlayer(1,inputLayers=[self.input_layer],activationFn="relu")
            self.output_layer = nlayer(output_shape, inputLayers=[hiddenLayer], isDynamic=1)
        else:
            self.output_layer = nlayer(output_shape, inputLayers=[self.input_layer], isDynamic=1)


    def addLayerAtLast(self, shape, isDynamic=1, activationFn="linear"):
        oldInputs = self.output_layer.input_layers
        newLayer = nlayer(shape=shape, inputLayers=oldInputs, isDynamic=isDynamic, activationFn=activationFn)
        newLayer.weights = self.output_layer.weights
        self.output_layer.weights = None
        self.output_layer.input_layers=[]
        self.output_layer.addInputLayer(newLayer)

        ## Transferring Weight matrix

    def setInput(self, input_values):
        # print("SETTING INPUT LAYER & STORING VALUES")
        if(type(self.input_layer) != type(None)):
            if(len(input_values) < self.input_layer.shape):
                print("ERROR: Unable to reduce input layer shape. Insert len(input values) >= input_shape")
            else:
                self.input_layer.shape = len(input_values)
                self.input_layer.cachedRun = -1
                self.input_layer.isDynamic = 0
                self.input_layer.cacheValue = np.array(input_values)
        else:   ## Initialize new input layer
                self.input_layer = nlayer(len(input_values), isInput=1, setInputValues=np.array(input_values))

                
                linker = self.output_layer
                if(type(linker) != type(None)):
                    while(len(linker.input_layers) > 0):    ## following only oldest (1st in list) links to reach input
                        linker = linker.input_layers[0]                               
                    linker.input_layer = [self.input_layer]



    def forward_prop(self, input_values=None):    # find result activation from input activation and weights
        global runNum
        if(type(input_values) != type(None)):
            self.input_layer.cacheValue = input_values

        if(self.input_layer.cachedRun == -1 and type(self.input_layer.cacheValue) != type(None)):
            output_activations = self.output_layer.getActivation()
            runNum += 1
            return output_activations
        else:
            print("Input uninitialized")
            return -1

    def backward_prop(self, input_values, trueOutput):
        self.input_layer.cacheValue = input_values

        predictedOutput = self.output_layer.getActivation()
        
        return self.output_layer.correct_error(trueOutput - predictedOutput)


# TESTING

In [333]:
nt = network(784, 10, insertDefault=0)

In [334]:
nt.addLayerAtLast(10,isDynamic=0,activationFn="relu")
nt.addLayerAtLast(10,isDynamic=0,activationFn="relu")

In [335]:
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('mnist-train.csv')

In [353]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

X_trainT = X_train.T

def one_hot(Y, maxExpected):
    one_hot_Y = np.zeros((Y.size, maxExpected + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

## find the most probable guessed by network
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


In [354]:

telementary = 1

In [355]:
len(X_trainT)

41000

In [356]:
X_train.T[0].shape

(784,)

In [357]:
runNum = 120

In [358]:
print(nt.forward_prop(X_trainT[1]))

Provided input from cache
activation = [43.546764   53.10551581 47.09198547 49.83707745 45.33959999 48.93422512
 49.12711997 45.77491603 43.69927044 51.34468906] & cached
activation = [252.09332699 242.82509197 256.76448106 185.45450935 191.50785967
 341.27766395 229.05875157 176.65125559 180.06905017 257.47177331] & cached
activation = [ 885.94460699 1223.12284865 1173.48071959 1379.26932166  984.12192475
  970.73582389 1396.2793819  1085.62100297 1136.85461841 1460.29327339] & cached
[ 885.94460699 1223.12284865 1173.48071959 1379.26932166  984.12192475
  970.73582389 1396.2793819  1085.62100297 1136.85461841 1460.29327339]


In [359]:
# nt.output_layer.cachedRun
nt.output_layer.input_layers[0].cachedRun

120

In [364]:
Y_train[0]

9

In [365]:
one_hot_yTEST = one_hot(Y_train[0], 9)
print(one_hot_yTEST) 

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]]


In [366]:
accuracySum = 0
runCount = 0

for it in range(len(X_trainT)):
    one_hot_y = one_hot(Y_train[it], 9)
    pred = get_predictions(nt.backward_prop(X_trainT[it],one_hot_y))
    if(pred.all() == one_hot_y.all()):
        accuracySum += 1
    runCount += 1
    
# accuracy = accuracySum/len(X_train)
accuracy = accuracySum/runCount



Provided input from cache
activation = [34.85892915 43.17644113 37.71560267 41.87985868 38.88328244 34.79771956
 40.01467537 40.64950868 33.69887808 38.73568654] & cached
activation = [203.02878551 194.22660707 207.51177633 148.18399825 156.46385964
 271.9075824  185.72602105 141.0139782  145.01433635 208.57230914] & cached
activation = [ 713.88356574  986.24371744  945.9522797  1109.41114889  792.47459727
  780.84847419 1122.97428945  875.75381593  916.99325241 1173.94844948] & cached
Re-used Cached Value


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 10)

In [269]:
accuracy

0.8979591836734694

In [256]:
n1 = network(3,1, insertDefault=1)

In [257]:
n1.setInput([3,2,1])

In [258]:
n1.output_layer.input_layers[0].input_layers[0].cacheValue

array([3, 2, 1])

In [241]:
print(n1.output_layer.input_layers[0].cachedRun)

-2


In [242]:
n1.setInput([3,1,2])

In [243]:
print(n1.forward_prop())

[0.73495495]


In [244]:
xIn = np.array([1.0,2.0,3.0])
yOut = np.array([0.0])

In [245]:
n1.output_layer.weights

array([[0.16821053]])

In [246]:
print(n1.backward_prop(xIn, yOut))

None


In [247]:
telementary = 0

In [248]:
for j in range(20):
    for i in range(10000):
        n1.backward_prop(xIn, yOut)

    print(n1.forward_prop(xIn))




[0.71219564]
[-0.14245337]
[-0.1139627]
[-0.09117016]
[-0.07293613]
[-0.0583489]
[-0.04667912]
[-0.0373433]
[-0.02987464]
[-0.02389971]
[-0.01911977]
[-0.01529581]
[-0.01223665]
[-0.00978932]
[-0.00783146]
[-0.00626517]
[-0.00501213]
[-0.00400971]
[-0.00320776]
[-0.00256621]


In [202]:
n1.forward_prop(xIn)

array([4.4012041])

layerTesting

In [14]:
inLay = nlayer(2, isInput=1, setInputValues=[2,3])

hidLay = nlayer(1, inputLayers=[inLay], activationFn="relu")

In [15]:
# hidLay.bias
# hidLay.weights
# hidLay.cacheValue
hidLay.input_layers
# inLay.input_layers

In [16]:
hidLay.getActivation()

activation = [0.97440479] & cached


array([0.97440479])

In [17]:
newInLay = nlayer(3,isInput=1,setInputValues=[2,3,5])

In [10]:
hidLay.addInputLayer(newInputLayer=newInLay)

In [11]:
hidLay.weights

array([[0.99885793, 0.9725909 , 0.72486948, 0.35375928, 0.07909738]])

In [12]:
hidLay.addWidth_to_Layer(2)

In [13]:
hidLay.weights

array([[0.99885793, 0.9725909 , 0.72486948, 0.35375928, 0.07909738],
       [0.10659773, 0.55853347, 0.62134468, 0.07597125, 0.98782191],
       [0.06992685, 0.24441412, 0.73520054, 0.26422487, 0.89743243]])

In [14]:
runNum += 1

In [28]:
hidLay.getActivation()

array([5.99525239, 6.29679149, 7.71707944])

# Playground

In [26]:
layerLengths = [2,3,1]
        
        ## Splitting input corrections to their corresponding layers
splitPoints = []
lengthTillNow = 0
for length in layerLengths:
    lengthTillNow += length
    splitPoints.append(lengthTillNow)

In [32]:
## Splitting input corrections to their corresponding layers

layerLengths = [2,3,1]
dIZ = range(6)
splitPoints = [0]
lengthTillNow = 0
for layerIndx in range(len(layerLengths)):
    lengthTillNow += layerLengths[layerIndx]
    splitPoints.append(lengthTillNow)

    print(dIZ[splitPoints[-2]:splitPoints[-1]])


range(0, 2)
range(2, 5)
range(5, 6)


In [27]:
splitPoints

[2, 5, 6]

In [60]:
x = np.array([[2,3,4]])
# x = np.array([2,3,4])
# x[0:2]

array([[2, 3, 4]])

In [61]:
len(x.shape)

2

In [67]:
x

array([[2, 3, 4]])

In [62]:
# x = x[np.newaxis]

In [65]:
x.T

array([[2],
       [3],
       [4]])

In [66]:
y = np.array([[1]])

print(np.matmul(y, x.T))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)

In [81]:
dZ = np.array([0.25512582])
inputArr = np.array([[2.0,3.0,4.0]])
np.matmul(dZ,inputArr)


array([0.51025164, 0.76537746, 1.02050328])

In [74]:
dZ.shape

(1,)

In [75]:
inputArr.shape

(1, 3)

In [78]:
np.matmul(dZ,inputArr)

array([0.51025164, 0.76537746, 1.02050328])

In [95]:
# empArr = np.empty(1,dtype=float)
empArr = np.array([4,],dtype=float)
inArr = np.array([1,2],dtype=float)
# inArr = np.

In [96]:
print(inArr)

[1. 2.]


In [109]:
empArr = np.concatenate((empArr, inArr))
print(empArr)
print(empArr.shape)
# print(newArr)


[7. 1. 2. 1. 2.]
(5,)


In [123]:
wts = np.array([[3,2,1],[1,0,-1]],dtype=float)

In [107]:
print("wts", wts.shape)
print("emp", empArr.shape)
act = np.matmul(wts,empArr)
print("act", act)

wts (2, 3)
emp (3,)
act [25.  5.]


In [110]:
empArr

array([7., 1., 2., 1., 2.])

In [117]:
# columns to add = empArr.shape[0] - wts.shape[1]
newCols = np.random.rand(wts.shape[0], empArr.shape[0] - wts.shape[1])

In [124]:
wts

array([[ 3.,  2.,  1.],
       [ 1.,  0., -1.]])

In [126]:
wts = np.concatenate((wts, newCols), axis=1)

In [94]:
np.random.rand(2,3)

array([[0.21734973, 0.22785161, 0.77439285],
       [0.26069918, 0.76749742, 0.33375012]])

In [30]:
x = np.array([1,2])

In [32]:
y = np.array(x)

In [163]:
x = [2,3]
y = x.append(5)

In [164]:
print(x)

[2, 3, 5]


In [19]:
a = np.array([[1, 0], [0, 1]])
b = np.array([[4, 1], [2, 2]])
a.dot(b.T)

array([[4, 2],
       [1, 2]])

In [20]:
a.dot(b)

array([[4, 1],
       [2, 2]])

In [22]:
np.matmul(a,b)

array([[4, 1],
       [2, 2]])

# MAIN

In [5]:
n1 = network(2,1)

in1 = np.array([0,1,2])
wtMat = np.array([[5,6,7],[8,9,10]])
# biases = np.array([5,25])
biases = np.array([0.5,0.25])

In [37]:
output_activations = np.matmul(wtMat, in1) + biases
print(output_activations)

[20.5  29.25]


In [38]:
n1.forward_prop(input_activations=in1, weight_matrix=wtMat, bias_ndarrray=biases)

array([20.5 , 29.25])